# Aquarium (水箱)

> The rules of Aquarium are simple:

- The puzzle is played on a rectangular grid divided into blocks called "aquariums"
- You have to "fill" the aquariums with water up to a certain level or leave it empty.
- The water level in each aquarium is one and the same across its full width
- The numbers outside the grid show the number of filled cells horizontally and vertically. 

-----------

> 规则：

1. 游戏棋盘被分成几块，每块被称为一个“水箱”；
2. 游戏中你可以给每个水箱灌一些水，也可以让它空着；
3. 同一个水箱内的水位是等高的。也即同一水箱内的同一行的单元格，要么都有水，要么都空着；
4. 棋盘外面的数字是该行或该列，灌水的单元格总数。 

In [21]:
from ortools.sat.python import cp_model as cp 

def AquariumSolver(X, Y, grid):
    cell_dict = dict()
    
    model = cp.CpModel()
    solver = cp.CpSolver()
    
    row_nums = {}
    # col_nums = {}
    
    x = {}
    X = list(map(lambda x: int(x), X.split(",")))
    Y = list(map(lambda x: int(x), Y.split(",")))
    
    X_ = len(X)
    Y_ = len(Y)
    
    for idx, char_ in enumerate(grid):
        if char_ not in cell_dict:
            cell_dict[char_] = []
            row_nums[char_] = []
        cell_dict[char_].append((idx // Y_, idx % X_))
    
    for i in range(0, len(grid), X_):
        row_ = grid[i: i + X_]
        for key in cell_dict.keys():
            row_nums[key].append(row_.count(key))
    
    # for i in range(Y):
    #     col_ = grid[i::X]
    #     for key in cell_dict.keys():
    #         col_nums[key].append(col_.count(key))
    
    for i in range(Y_):
        for j in cell_dict.keys():
            x[i, j] =  model.NewBoolVar(name = f"x[{i}, {j}]")
    
    for idx, val in enumerate(X):
        col_ = grid[idx: : X_]
        current_col_variables = []
        for idx_temp in range(len(col_)):
            current_col_variables.append(x[idx_temp, col_[idx_temp]])
        model.Add(sum(current_col_variables) == val)
    
    for idx in range(Y_):
        current_row_variables = []
        for char_ in cell_dict.keys():
            if row_nums[char_][idx] == 0:
                x[idx, char_] = 0
            current_row_variables.append(row_nums[char_][idx] * x[idx, char_])
        model.Add(sum(current_row_variables) == Y[idx])
            
    
    status = solver.Solve(model)
    if status == cp.OPTIMAL:
        print("FOUND OPTIMAL")
        outputline = ""
        for idx, char_ in enumerate(grid):
            if solver.Value(x[idx // X_, char_]) == 1:
                outputline += "# "
            else:
                outputline += ". "
            if idx % X_ == X_ - 1:
                print(outputline)
                outputline = ""


if __name__ == "__main__":
    # AquariumSolver(
    #     X = "9,9,7,8,6,4,4,5,6,4", 
    #     Y = "2,5,6,6,7,6,4,9,8,9", 
    #     grid = "AAABBCCCCCDDAABBBCCCAAAAAEBCFGAAHHEEECFGIIHHJJECGGKKHKKEECLMKKNJNOPPLMKNNNNOPPPMKNKOOOQRPPKKKSSOQRTT")
    # AquariumSolver(
    #     X = "3,2,2,2,3,4", 
    #     Y = "3,1,3,2,3,4", 
    #     grid = "AAABBBAAAAABACCDBBCCDDBBCCDEFFEEEEFF")
    
    AquariumSolver(
        X = "7,6,7,5,6,6,5,4,3,4",
        Y = "1,6,3,4,6,5,5,7,8,8",
        grid = "ABCCCCDDDEABFFFCCDDEGBHHIJJKDDGGLLLMMKDNGOPPQQMDDNRRSPQTTUUUVSSWXYYUUUZaWWXXYYYbaaccXYYYddeeeeeeffgg"
    )
    

FOUND OPTIMAL
# . . . . . . . . . 
# # # # # . . . . # 
. . . . . . . # # # 
# # . . . . . # # . 
# . # # # # # . . . 
# # # . . # # . . . 
. # # . # # # . . . 
# # # # # # . . . # 
. . # # # # # # # # 
# # # # # # # # . . 


![](../assets/figures/Aquarium1.png)

![](../assets/figures/Aquarium2.png)